In [1]:
import time
import statistics as stats
import multiprocessing
import time 
import random
import time
from tqdm import tqdm
from datetime import timedelta
from datetime import datetime
start_program = time.time()
from multiprocessing import *
import pandas as pd
import numpy as np
from threading import Thread
import time
import subprocess 


In [2]:
results = [[] for i in range(5)]

n_it = 100


n_st = number_stations = 10
n_s = number_sensors = 100

def_st = 1
def_s = 3
def_r = 'day'


set_date = [random.random() for i in range(500)]


In [3]:

def str_time_prop(start, end, time_format, prop):
    """Get a time at a proportion of a range of two formatted times.

    start and end should be strings specifying times formatted in the
    given format (strftime-style), giving an interval [start, end].
    prop specifies how a proportion of the interval to be taken after
    start.  The returned time will be in the specified format.
    """

    stime = time.mktime(time.strptime(start, time_format))
    etime = time.mktime(time.strptime(end, time_format))

    ptime = stime + prop * (etime - stime)

    return time.strftime(time_format, time.localtime(ptime))



def random_date(start, end, prop, dform = '%Y-%m-%dT%H:%M:%S'):
    return str_time_prop(start, end, dform, prop)
    
    
def to_pm(v):
    return str(round(v[0][0],2)) + "$" + '\\' + "pm$" + str(round(v[1][0],2))


# Queries

In [4]:
# A simple class
# attribute
# e_q1 = """select seq_search(t,<timestamp> - <nb> * <rangesUnit>,<timestamp>) as tt, s<sid>@tt FROM d1_v WHERE id_station = 'st<stid>';"""
e_q1 = """select id_station, seq_search(t,<timestamp> - <nb> * <rangesUnit>,<timestamp>) as tt, <sid> FROM d1_v WHERE id_station in <stid>;"""


e_q2 = """select id_station, tfe, <sidlist> from (
select id_station, seq_search(t,<timestamp> - <nb> * <rangesUnit>,<timestamp>) as tt, <sfilter> as fe, t@fe as tfe, <sid_filtered> FROM d1_v WHERE id_station in <stid>
); """ #!seq_filter_search(s<sid>@tt > 0.95, tt),. <sid>@fe 

e_q2AND = """select id_station, seq_search(t,<timestamp> - <nb> * <rangesUnit>,<timestamp>) as tt, <sfilterAND> as fe, <sid_filtered> FROM d1_v WHERE id_station in <stid>; """ #!seq_filter_search(s<sid>@tt > 0.95, tt),. <sid>@fe 


e_q3 = """SELECT id_station, ! seq_search(t,<timestamp> - <nb> * <rangesUnit>,<timestamp>) as tt, <seq_avg> FROM d1_v WHERE id_station in <stid>;""" # where id_station = 'st<stid>', <seq_avg>(<sid>@tt)


e_q4 = """select id_station, ts, <sidlist> from (
select id_station, seq_search(t,<timestamp> - <nb> * <rangesUnit>, <timestamp>) as tt, seq_group_agg_avg(t@tt , t@tt/3600) as ts, <seq_group_agg_avg> FROM d1_v where id_station in <stid>
);""" #seq_group_agg_avg(s<sid>@tt, t@tt/3600) 


e_q5 = """select seq_aprogres_datetime(<timestamp> -  <nb> * <rangesUnit>, 5, <nb> * <rangesUnit>) as ts5, <seq_stretch> from d1_v where id_station in <stid>;""" #seq_stretch(ts5,t,s<sid>)



In [5]:
import pymonetdb
from tqdm import tqdm 
import time

class EXtremeDB:
 
    # A sample method 
    @staticmethod
    def query(query, db, max_d, rangesUnit, n_it, start_date = "2019-04-01 00:00:00", stop_date = "2019-04-30 00:00:00", n_st = 1, n_s = 10):
        
        import exdb 
        import datetime
        exdb.init_runtime(debug = False, shm = False, disk = False, tmgr = 'mursiw')
        con = exdb.connect('diufrm118', 5001)
        curs = con.cursor()
        curs.execute("SELECT s23 FROM d1_v where id_station = 'st3'")
        curs.fetchall()
        results = [[],[]]
        options = {"day" : 60 * 60* 24,
                   "week" : 60 * 60* 24 * 7,
                   "minute" : 60,
                   "hour" : 60 * 60,
                   "second" : 1,
                   "month" : 60 * 60 * 24 * 30,
                   "year" :  60 * 60 * 24 * 30 * 12
        }
        duration = max_d
        runtimes = []
        full_time = time.time()
        for i in tqdm(range(n_it)):
#             time.sleep(0.5)
            date = random_date(start_date, stop_date, set_date[(duration*i)%500], dform = '%Y-%m-%d %H:%M:%S')
            date = int(time.mktime(datetime.datetime.strptime(date, '%Y-%m-%d %H:%M:%S').timetuple()))
            temp = query.replace("<timestamp>", str(date))
            temp = temp.replace("<nb>", str(duration))
#             temp = temp.replace("<sid>", str(set_s[(duration*i)%500]))
            temp = temp.replace("<rangesUnit>", str(options[rangesUnit]))
        
            li = ['st' + str(z) for z in random.sample(range(number_stations), n_st)]
            q = '(' + "'" + li[0] + "'"
            for j in li[1:]:
                q += ', ' + "'" + j + "'"
            q += ")"
            temp = temp.replace("<stid>", q)
            temp = temp.replace("<db>", db)
            
            # sensor
            rand = [str(z) for z in random.sample(range(number_sensors), n_s)]
#             print(rand)
            sidlist = 's' + rand[0]
            for j in rand[1:]:
                sidlist += ',' + 's' +  j
#             print(sidlist)
            li = ['s' + str(z) + "@tt" for z in rand]
            li_filtered = ['s' + str(z) + "@fe as s" + str(z) for z in rand]
#             <seq_group_agg_avg> seq_group_agg_avg(s<sid>@tt, t@tt/3600), 
#             <seq_stretch> #seq_stretch(ts5,t,s<sid>), 
#             <sfilter> !seq_filter_search(s<sid>@tt > 0.95, tt)
            
            q = li[0]
            q_filtered = li_filtered[0] 
            q_seq_group_agg_avg = "seq_group_agg_avg(" + li[0] + " , t@tt/3600) as " + li[0].split('@')[0]
            q_seq_avg = "seq_avg(" + li[0] + ")" 
            q_seq_stretch = "seq_stretch(ts5,t," + li[0].split('@')[0] + ")" 
#             q_filter = "!seq_filter_search(" +li[0] + "> 0.95, tt)"
            q_filter = "!seq_filter_search(" +li[0] + "> 0.95"
            q_filterAND = "!seq_filter_search(" +li[0] + "> 0.95"
            
#             q_avg = 'avg(' + li[0] + ')'
            for j in range(1,len(li_filtered)):
                q_filtered += ', ' + li_filtered[j] 
    
            for j in li[1:]:
                q += ', ' + j
                q_seq_avg += ", seq_avg(" + j + ")" 
                q_seq_group_agg_avg += ", seq_group_agg_avg(" + j + " , t@tt/3600)" + " as " +  j.split('@')[0] #        li[0] + ' > 0.95'
                q_seq_stretch += ", seq_stretch(ts5,t," + j.split('@')[0] + ")" 
# #                 q_filter += ", !seq_filter_search(" + j + "> 0.95, tt)"
#                 q_filter += " OR " + j + "> 0.95"
#                 q_filterAND += " AND " + j + "> 0.95"
# #                 q_filter += ' OR ' + j + ' > 0.95'
# #                 q_avg += ', ' + 'avg(' + j + ')'
            temp = temp.replace("<sid>", q)
            temp = temp.replace("<sidlist>", sidlist)
            temp = temp.replace("<seq_avg>", q_seq_avg)
            temp = temp.replace("<sid_filtered>", q_filtered)
            temp = temp.replace("<seq_group_agg_avg>", q_seq_group_agg_avg)
            temp = temp.replace("<sfilter>", q_filter + ", tt)")
            temp = temp.replace("<sfilterAND>", q_filterAND + ", tt)")
            temp = temp.replace("<seq_stretch>", q_seq_stretch)
#             temp = temp.replace("<avg_s>", q_avg)            
        
#             print(temp)
            start = time.time()
            curs.execute(temp)
            curs.fetchall()
            # print(curs.rowcount)
#             print(curs.fetchall()[0])
            diff = (time.time()-start)*1000
#                 print(temp, diff)
            runtimes.append(diff)
            if time.time() - full_time > 20 and i > 5: 
                break  
#         print(runtimes)
#         print(temp)
        results[0].append(stats.mean(runtimes))
#             results[1].append(percentile(runtimes,85))
        results[1].append(stats.stdev(runtimes))
        con.close()
        return results

    
    
    @staticmethod
    def insert_file(batch_size,rate, st):
#         template_sql = "INSERT INTO d1_wide_slow_ingest (time, id_station"
#         for i in range(100):
#             template_sql += ", s" + str(i)
#         template_sql += ") VALUES ('%s', '%s',"  + ','.join(100* [' %s']) + ');'

        import exdb 
        import datetime
        exdb.init_runtime(debug = False, shm = False, disk = False, tmgr = 'mursiw')
        conn = exdb.connect('diufrm118', 5001)
        cur = conn.cursor()
        cur.execute("set append_mode true")
        file = '/home/abdel/venv2/bin/insertion-100/short_ts' + str(st) + '.csv'
        f = open(file, "r")
        f = f.read()
        f = f.splitlines()[1:]
        for i in range(len(f)):
            f[i] = f[i].split(',')            
            temp = "'" + f[i][1] + "' , " + "'" + ' '.join(f[i][0].split('T')) + "' "
            for e in f[i][2:]:
                temp += ", " + str(e)
            f[i] = temp
        queries = []
        q_template = "INSERT or UPDATE INTO d1_ingest_v (id_station, t"
        for k in range(100):
            q_template += ", s" + str(k)
        q_template += ") VALUES ("
        for i in range(0, len(f), rate):
            q = q_template +  f[i] + ")"
            for i in f[i+1:i+rate]:
                q += ", (" + i + ")"
            queries.append(q)
        start = time.time()
        a = 0
        print('insertion st' + str(st) + ' started')
        while not stop_insertion[st]: 
            starttime = time.time()
            try: 
                cur.execute(queries[a])
            except: 
                print(queries[a])
            a += 1
            diff = 1.0 - ((time.time() - starttime))
            try: 
                time.sleep(diff)
            except: 
                print(diff)
        
    @staticmethod
    def run_all(start, stop, db):
        results[0].append(to_pm(EXtremeDB.query(e_q1, db, 1, def_r, n_it, start, stop, n_st = def_st, n_s = def_s)))
        results[1].append(to_pm(EXtremeDB.query(e_q2, db, 1, def_r, n_it, start, stop, n_st = def_st, n_s = def_s)))
        results[2].append(to_pm(EXtremeDB.query(e_q3, db, 1, def_r, n_it, start, stop, n_st = def_st, n_s = def_s)))
        results[3].append(to_pm(EXtremeDB.query(e_q4, db, 1, def_r, n_it, start, stop, n_st = def_st, n_s = def_s)))
        results[4].append(to_pm(EXtremeDB.query(e_q5, db, 1, def_r, n_it, start, stop, n_st = def_st, n_s = def_s)))
    
    
    
    

In [6]:
EXtremeDB.run_all("2019-04-01 00:00:00", "2019-04-30 00:00:00", "d1_v")


 66%|██████████████████████▍           | 66/100 [00:20<00:10,  3.26it/s]


In [ ]:
n_threads = 100
stop_insertion = [False for i in range(n_threads)]
rate = 20

# 20K dp/s

# Launch insertion
for i in range(n_threads):  
    t1 = Thread(target = EXtremeDB.insert_file, args=(100_000, rate, i))
#     t1 = Thread(target = ClickHouse.insert_by_point, args=(rate, i))
    t1.setDaemon(True)
    t1.start()

# insert_new(200, 1)

In [7]:
rate = 20

In [8]:
# select count(*) from select flattened id_station, seq_search(t,1556582400,1559496000) as tt, s4 FROM d1_ingest_v;

In [14]:
time.sleep(8 * 8640 // rate)

In [10]:
EXtremeDB.run_all("2019-04-01 00:00:00", "2019-04-30 00:00:00", "d1_v_ingest")

 28%|█████████▌                        | 28/100 [00:20<00:52,  1.37it/s]


In [11]:
EXtremeDB.run_all("2019-05-01 01:00:00", "2019-05-02 00:00:00", "d1_v_ingest")

 31%|██████████▌                       | 31/100 [00:21<00:47,  1.46it/s]


In [12]:
EXtremeDB.run_all("2019-04-29 01:00:00", "2019-05-02 00:00:00", "d1_v_ingest")

 21%|███████▏                          | 21/100 [00:21<01:22,  1.04s/it]


In [ ]:
stop_insertion = [True for i in range(n_threads)]


In [13]:
results 

[['37.27$\\pm$7.78',
  '186.11$\\pm$596.16',
  '152.22$\\pm$693.73',
  '187.28$\\pm$504.03'],
 ['7.92$\\pm$7.36', '6.99$\\pm$5.82', '49.11$\\pm$427.8', '49.19$\\pm$292.84'],
 ['8.93$\\pm$8.45', '7.8$\\pm$6.37', '50.69$\\pm$342.34', '5.26$\\pm$3.78'],
 ['8.14$\\pm$8.08', '409.02$\\pm$1122.77', '5.31$\\pm$38.32', '2.2$\\pm$1.6'],
 ['301.25$\\pm$10.67',
  '703.2$\\pm$844.48',
  '661.11$\\pm$696.06',
  '994.67$\\pm$1325.66']]

In [ ]:
runtimes = [[float(results[i][j].split('$')[0]) for j in range(len(results[i]))][:4] for i in range(5)]
variance = [[float(results[i][j].split('$')[2]) for j in range(len(results[i]))][:4] for i in range(5)]

In [ ]:
runtimes = list(map(list, zip(*runtimes)))
variance = list(map(list, zip(*variance)))


In [ ]:
runtimes

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
X = np.arange(5)
labels = ['query'+str(1+i) for i in range(5)]
fig = plt.figure(figsize=(8, 6))
ax = fig.add_axes([0,0,1,1])
for i in range(4):
    ax.bar(X +0.15*i, runtimes[i], yerr = variance[i], width = 0.15)
ax.legend(['offline', 'online old', 'online new', 'online mixed', ])
ax.set_xticklabels(labels)
ax.set_xticks(np.arange(len(labels)))
ax.yaxis.grid(True)
plt.title('ClickHouse offline vs. online, rate: '+ str(rate) + ' x ' + str(n_threads))
plt.tight_layout()

# ax.bar(X+0.25, data[1], color = 'g', width = 0.25)
# ax.bar(X+0.5, data[2], color = 'r', width = 0.25)

In [ ]:
import exdb 
import datetime
exdb.init_runtime(debug = False, shm = False, disk = False, tmgr = 'mursiw')
con = exdb.connect('diufrm118', 5001)
curs = con.cursor()
curs.execute("DROP table d1_ingest_v;")
curs.execute("CREATE TABLE d1_ingest_v (id_station string PRIMARY KEY, t sequence(TIMESTAMP asc), s0 sequence(DOUBLE), s1 sequence(DOUBLE), s2 sequence(DOUBLE), s3 sequence(DOUBLE), s4 sequence(DOUBLE), s5 sequence(DOUBLE), s6 sequence(DOUBLE), s7 sequence(DOUBLE), s8 sequence(DOUBLE), s9 sequence(DOUBLE), s10 sequence(DOUBLE), s11 sequence(DOUBLE), s12 sequence(DOUBLE), s13 sequence(DOUBLE), s14 sequence(DOUBLE), s15 sequence(DOUBLE), s16 sequence(DOUBLE), s17 sequence(DOUBLE), s18 sequence(DOUBLE), s19 sequence(DOUBLE), s20 sequence(DOUBLE), s21 sequence(DOUBLE), s22 sequence(DOUBLE), s23 sequence(DOUBLE), s24 sequence(DOUBLE), s25 sequence(DOUBLE), s26 sequence(DOUBLE), s27 sequence(DOUBLE), s28 sequence(DOUBLE), s29 sequence(DOUBLE), s30 sequence(DOUBLE), s31 sequence(DOUBLE), s32 sequence(DOUBLE), s33 sequence(DOUBLE), s34 sequence(DOUBLE), s35 sequence(DOUBLE), s36 sequence(DOUBLE), s37 sequence(DOUBLE), s38 sequence(DOUBLE), s39 sequence(DOUBLE), s40 sequence(DOUBLE), s41 sequence(DOUBLE), s42 sequence(DOUBLE), s43 sequence(DOUBLE), s44 sequence(DOUBLE), s45 sequence(DOUBLE), s46 sequence(DOUBLE), s47 sequence(DOUBLE), s48 sequence(DOUBLE), s49 sequence(DOUBLE), s50 sequence(DOUBLE), s51 sequence(DOUBLE), s52 sequence(DOUBLE), s53 sequence(DOUBLE), s54 sequence(DOUBLE), s55 sequence(DOUBLE), s56 sequence(DOUBLE), s57 sequence(DOUBLE), s58 sequence(DOUBLE), s59 sequence(DOUBLE), s60 sequence(DOUBLE), s61 sequence(DOUBLE), s62 sequence(DOUBLE), s63 sequence(DOUBLE), s64 sequence(DOUBLE), s65 sequence(DOUBLE), s66 sequence(DOUBLE), s67 sequence(DOUBLE), s68 sequence(DOUBLE), s69 sequence(DOUBLE), s70 sequence(DOUBLE), s71 sequence(DOUBLE), s72 sequence(DOUBLE), s73 sequence(DOUBLE), s74 sequence(DOUBLE), s75 sequence(DOUBLE), s76 sequence(DOUBLE), s77 sequence(DOUBLE), s78 sequence(DOUBLE), s79 sequence(DOUBLE), s80 sequence(DOUBLE), s81 sequence(DOUBLE), s82 sequence(DOUBLE), s83 sequence(DOUBLE), s84 sequence(DOUBLE), s85 sequence(DOUBLE), s86 sequence(DOUBLE), s87 sequence(DOUBLE), s88 sequence(DOUBLE), s89 sequence(DOUBLE), s90 sequence(DOUBLE), s91 sequence(DOUBLE), s92 sequence(DOUBLE), s93 sequence(DOUBLE), s94 sequence(DOUBLE), s95 sequence(DOUBLE), s96 sequence(DOUBLE), s97 sequence(DOUBLE), s98 sequence(DOUBLE), s99 sequence(DOUBLE));")
# curs.execute("trace on")
curs.execute("insert or update into d1_ingest_v select id_station,(substr(t,1,10)||' '||substr(t,12,8))::timestamp::bigint,s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21,s22,s23,s24,s25,s26,s27,s28,s29,s30,s31,s32,s33,s34,s35,s36,s37,s38,s39,s40,s41,s42,s43,s44,s45,s46,s47,s48,s49,s50,s51,s52,s53,s54,s55,s56,s57,s58,s59,s60,s61,s62,s63,s64,s65,s66,s67,s68,s69,s70,s71,s72,s73,s74,s75,s76,s77,s78,s79,s80,s81,s82,s83,s84,s85,s86,s87,s88,s89,s90,s91,s92,s93,s94,s95,s96,s97,s98,s99 from foreign table(path='/home/abdel/d1_data/d1.csv', skip=1) as d1_h;")
        

In [ ]:
# insert data